In [1]:
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.interactions.pairwise import PairwiseCorr
from drn_interactions.io import load_neurons, load_derived_generic
import seaborn as sns
import pandas as pd
import numpy as np
from drn_interactions.interactions.graph_clustering import SpectralCluster, ClusterRunner, ClusterEvaluation, ClusterResults
import warnings
from drn_interactions.config import Config

In [2]:
def reorder_df_affinity(df, ensemble_labels):
    idx = np.argsort(ensemble_labels)
    return df.iloc[idx, idx]

def get_true(df, bin_width=1, shuffle=False):
    return df.query("bin_width == @bin_width and shuffle == @shuffle")

def get_labels_from_df_ensembles(df_ensembles, session):
    return (
        df_ensembles
        .query("session_name == @session")
        .sort_values("neuron_id")
        ["ensemble"]
        .values
    )

def get_affinity(session):
    loader = SpontaneousActivityLoader(bin_width=1, shuffle=False, session_name=session)
    preprocessor = InteractionsPreprocessor()
    pairwise = PairwiseCorr(rectify=True)

    spikes = preprocessor(loader())
    return pairwise.fit(spikes).get_adjacency_df()


In [3]:
sessions = load_neurons()["session_name"].unique()

preprocessor = InteractionsPreprocessor(z=True, minmax=False)
evaluator = ClusterEvaluation()

loader_fac = lambda bin_width, session, shuffle: SpontaneousActivityLoader(
    bin_width=bin_width, session_name=session, shuffle=shuffle
)
corr_fac = lambda: PairwiseCorr(
    remove_self_interactions=True,
    rectify=True,
    shuffle=False,
    remove_duplicate_combs=True,
)
cluster_fac = lambda: SpectralCluster(
    n_clusters=np.arange(2, 8), spectral_kws={"affinity": "precomputed"}, n_init=20
)

runner_fac = lambda bin_width, session, shuffle_load: ClusterRunner(
    loader=loader_fac(bin_width, session, shuffle_load),
    preprocessor=preprocessor,
    affinity_calculator=corr_fac(),
    clusterer=cluster_fac(),
    evalulator=evaluator,
)

### Dectect Ensembles at Multiple Timescales & Compare to Shuffle

In [4]:
ensemble_frames = []
ensemble_stats_frames = []
current_max = 1
bin_widths = (0.05, 0.1, 0.5, 1, 2)
nboot = 50

for shuffle in (False, True):
    for bin_width in bin_widths:
        for session in sessions:

            # calculate ensembles
            runner = runner_fac(bin_width, session, shuffle)
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                _, ensembles, _, ensemble_stats = runner.run(20)

            # update ensembles to global max
            ensembles["ensemble"] = np.where(
                ensembles["ensemble"] != -1, ensembles["ensemble"] + current_max, -1
            )
            ensemble_stats["ensemble"] = np.where(
                ensemble_stats["ensemble"] != -1,
                ensemble_stats["ensemble"] + current_max,
                -1,
            )
            current_max = ensembles["ensemble"].abs().max() + 1

            # add metadata and append to list of results
            ensembles = ensembles.assign(
                session_name=session, shuffle=shuffle, bin_width=bin_width
            )
            ensemble_stats = ensemble_stats.assign(
                session_name=session, shuffle=shuffle, bin_width=bin_width
            )
            ensemble_stats_frames.append(ensemble_stats)
            ensemble_frames.append(ensembles)


df_ensembles = pd.concat(ensemble_frames).reset_index(drop=True)
df_ensembles_stats = pd.concat(ensemble_stats_frames).reset_index(drop=True)

dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)
df_ensembles.to_csv(dd / "spont - ensembles.csv", index=False)
df_ensembles_stats.to_csv(dd / "spont - stats.csv", index=False)

### Subset Out 1 Sec, Non-Shuffled Ensembles & Calculate Descriptive Status

In [ ]:
# make true df_ensemble and df_ensembles_stats and global ensemble ID

partition_frames = []
stats_frames = []
evalulater = ClusterEvaluation()
true_ensembles = df_ensembles.pipe(get_true)
for session in sessions:
    labs = get_labels_from_df_ensembles(true_ensembles, session)
    df_affinity = get_affinity(session)
    partition_frames.append(
        evalulater.evaluate_partition(df_affinity, labs)
        .assign(session_name=session)
    )
    stats_frames.append(
        evalulater.evaluate_communities(df_affinity, labs)
        .assign(session_name=session)
    )

stats = pd.concat(stats_frames).reset_index(drop=True)
stats["ensemble_id"] = pd.factorize(stats["community"].astype(str) + stats["session_name"])[0]
stats["ensemble_id"] = np.where(
    stats["community"] == -1,
    -1,
    stats["ensemble_id"],
)
true_ensembles = (
    stats
    .rename(columns=dict(community="ensemble"))
    [["session_name", "ensemble", "ensemble_id"]]
    .merge(true_ensembles, on=["session_name", "ensemble"])
)


neuron_types = load_derived_generic("neuron_types.csv")
descriptor = ClusterResults(
    neuron_types=neuron_types
)
stats = descriptor.ensemble_results(true_ensembles, stats)


In [ ]:
# response entropy

slow_responders_shock = load_derived_generic(
    "slow_ts_foot_shock_unit_responders_pre_to_shock.csv"
)[["neuron_id", "diff_inv", "sig"]].assign(
    response=lambda x: np.where(
        x["sig"] == False,
        "no_response",
        np.where(x["diff_inv"] < 0, "inhibited", "activated"),
    )
)
fast_responders = load_derived_generic("fast_fs_foot_shock_unit_responders.csv")[
    ["neuron_id", "Diff", "sig"]
].assign(
    response=lambda x: np.where(
        x["sig"] == False,
        "no_response",
        np.where(x["Diff"] < 0, "inhibited", "activated"),
    )
)

bs_response = load_derived_generic("brain_states_spikerate_responders.csv")[
    ["neuron_id", "Diff", "sig"]
].assign(
    response=lambda x: np.where(
        x["sig"] == False,
        "no_response",
        np.where(x["Diff"] < 0, "inhibited", "activated"),
    )
)


res_slow = descriptor.ensemble_response_entropy(
    true_ensembles,
    slow_responders_shock,
    created_col="fs_slow_entropy",
)
fast_responders = descriptor.ensemble_response_entropy(
    true_ensembles,
    slow_responders_shock,
    created_col="fs_fast_entropy",
)
res_bs = descriptor.ensemble_response_entropy(
    true_ensembles,
    bs_response,
    created_col="bs_entropy",
)
response_entropy = res_slow.merge(fast_responders, how="outer").merge(
    res_bs, how="outer"
)

stats = stats.merge(response_entropy, how="outer")

In [ ]:
dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)
true_ensembles.to_csv(dd / "spont - ensembles - true.csv", index=False)
stats.to_csv(dd / "spont - stats - true.csv", index=False)

### Ensemble Props

In [11]:
dd = Config.derived_data_dir / "ensembles"
dd.mkdir(exist_ok=True)


neuron_types = load_derived_generic("neuron_types.csv")
df_ensembles = (
    pd.read_csv(dd / "spont - ensembles - true.csv")
    .merge(neuron_types[["neuron_id", "neuron_type"]])
)
stats = pd.read_csv(dd / "spont - stats - true.csv")

In [32]:
from drn_interactions.stats import se_prop
from scipy.stats import chi2_contingency


df_ensembles["in_ensemble"] = np.where(
    df_ensembles["ensemble"] != -1, 1, 0
    )

prop = df_ensembles["in_ensemble"].mean() * 100
se = se_prop(df_ensembles["in_ensemble"]) * 200

print(f"Proportion of units in an ensemble {prop:.2f}% +- {se:.2f}")

X = pd.crosstab(
    df_ensembles["in_ensemble"], df_ensembles["neuron_type"]
    )

stat, p, _, _ = chi2_contingency(X)

print(f"stat = {stat:.2f}.\np = {p:.2f}")

display(X)

Xt = pd.crosstab(
    df_ensembles["in_ensemble"], df_ensembles["neuron_type"], normalize="index"
    )
display(Xt.multiply(100).round(2))

Proportion of units in an ensemble 82.56% +- 3.20
stat = 3.99.
p = 0.14


neuron_type,FF,SIR,SR
in_ensemble,,,
0,23,35,40
1,71,192,201


neuron_type,FF,SIR,SR
in_ensemble,,,
0,23.47,35.71,40.82
1,15.30,41.38,43.32


In [44]:
from drn_interactions.stats import se_mean

def mean_se(x, round=2):
    return f"{x.mean().round(round)} +- {se_mean(x).round(2)}"

sig_ensembles = stats.query("ensemble_id != -1")

cont_cols = ["size", "average_weight", "conductance", "prop_total", "edge_expantion", "neuron_type_entropy"]

df = stats[cont_cols]
df.apply(mean_se).to_frame("Mean +- SE")

,Mean +- SE
size,11.17 +- 1.09
average_weight,0.31 +- 0.02
conductance,0.41 +- 0.02
prop_total,0.38 +- 0.03
edge_expantion,1.55 +- 0.28
neuron_type_entropy,1.11 +- 0.06
